In [1]:
import pandas as pd
import requests
import gmaps
from config import g_key


In [2]:
city_data_df=pd.read_csv("WeatherPy_database.csv")

In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain(inches)           float64
Snow(inches)           float64
dtype: object

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
min_temp= float(input("What is the minimum temperature you would like for your vacation"))
max_temp= float(input("What is the maximum temperature you would like for your vacation"))

What is the minimum temperature you would like for your vacation70
What is the maximum temperature you would like for your vacation90


In [6]:
rain_amount=input("Do you want it to be raining (yes/no)")


Do you want it to be raining (yes/no)no


In [7]:
snow_amount=input("Do you want it to be snowing (yes/no)")

Do you want it to be snowing (yes/no)no


In [8]:
if rain_amount == "no" and snow_amount== "no":
    preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) &
                                         (city_data_df["Max Temp"]>= min_temp)&
                                         (city_data_df["Rain(inches)"]== 0) &
                                         (city_data_df["Snow(inches)"]==0)]
elif rain_amount == "no" and snow_amount =="yes":
    preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) &
                                         (city_data_df["Max Temp"]>= min_temp)&
                                         (city_data_df["Rain(inches)"]== 0) &
                                         (city_data_df["Snow(inches)"]>=0)]
elif rain_amount == "yes" and snow_amount=="no":
    preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) &
                                         (city_data_df["Max Temp"]>= min_temp)&
                                         (city_data_df["Rain(inches)"]> 0) &
                                         (city_data_df["Snow(inches)"]==0)]
else:
    preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) &
                                         (city_data_df["Max Temp"]>= min_temp)&
                                         (city_data_df["Rain(inches)"]> 0) &
                                         (city_data_df["Snow(inches)"]>0)]
                                        
                                        

In [9]:
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(inches),Snow(inches)
1,1,Bubaque,GW,11.28,-15.83,82.15,78,75,11.23,broken clouds,0.0,0.0
6,6,Salalah,OM,17.02,54.09,84.20,83,20,1.12,few clouds,0.0,0.0
8,8,Samut Prakan,TH,13.60,100.60,86.00,89,20,9.17,light rain,0.0,0.0
9,9,Bukama,CD,-9.20,25.85,71.89,41,63,1.57,broken clouds,0.0,0.0
10,10,Quatre Cocos,MU,-20.21,57.76,72.00,68,20,16.11,few clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
563,563,Gatesville,US,31.44,-97.74,90.00,55,75,9.17,broken clouds,0.0,0.0
571,571,Marsa Matruh,EG,31.35,27.25,77.00,44,0,5.82,clear sky,0.0,0.0
572,572,Iralaya,HN,15.00,-83.23,85.08,68,73,11.32,broken clouds,0.0,0.0
573,573,Husainabad,IN,24.53,84.02,85.32,73,72,3.96,broken clouds,0.0,0.0


In [10]:
preferred_cities_df= preferred_cities_df.dropna()

In [11]:
preferred_cities_df.count()

City_ID                232
City                   232
Country                232
Lat                    232
Lng                    232
Max Temp               232
Humidity               232
Cloudiness             232
Wind Speed             232
Current Description    232
Rain(inches)           232
Snow(inches)           232
dtype: int64

In [20]:
hotel_df=preferred_cities_df[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()
hotel_df["Hotel Name"]= ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bubaque,GW,82.15,broken clouds,11.28,-15.83,
6,Salalah,OM,84.20,few clouds,17.02,54.09,
8,Samut Prakan,TH,86.00,light rain,13.60,100.60,
9,Bukama,CD,71.89,broken clouds,-9.20,25.85,
10,Quatre Cocos,MU,72.00,few clouds,-20.21,57.76,


In [29]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
  
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bubaque,GW,82.15,broken clouds,11.28,-15.83,Casa Dora
6,Salalah,OM,84.20,few clouds,17.02,54.09,HAMDAN PLAZA HOTEL
8,Samut Prakan,TH,86.00,light rain,13.60,100.60,The Color Living Hotel
9,Bukama,CD,71.89,broken clouds,-9.20,25.85,
10,Quatre Cocos,MU,72.00,few clouds,-20.21,57.76,Veranda Palmar Beach Hotel


In [31]:
output_data_file="WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
vacation_df=pd.read_csv("WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Bubaque,GW,82.15,broken clouds,11.28,-15.83,Casa Dora
1,6,Salalah,OM,84.20,few clouds,17.02,54.09,HAMDAN PLAZA HOTEL
2,8,Samut Prakan,TH,86.00,light rain,13.60,100.60,The Color Living Hotel
3,9,Bukama,CD,71.89,broken clouds,-9.20,25.85,NaN
4,10,Quatre Cocos,MU,72.00,few clouds,-20.21,57.76,Veranda Palmar Beach Hotel


In [39]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} at {Max Temp} °F</dd>
</dl>
"""


In [40]:

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations=vacation_df[["Lat", "Lng"]]

In [42]:
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))